<h1> Generate simulator data </h1>

In [1]:
from pyfme.aircrafts import SimplifiedCessna172, BasisLinear, Debug
from pyfme.aircrafts.aircraft import Aircraft
import matplotlib.pyplot as plt
import numpy as np
import json
import pandas as pd
import quaternion

In [2]:
# aircraft = SimplifiedCessna172()
aircraft = BasisLinear('../src/pyfme/aircrafts/basis_linear.json')
# aircraft = Debug()

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from pyfme.environment import Environment
from pyfme.models import RigidBodyEuler, RigidBodyEulerState, RigidBodyQuat, RigidBodyQuatState
from pyfme.aircrafts.aircraft import ConventionalControls
from pyfme.utils.input_generator import Constant, Step
import time

In [5]:
from pyfme.models.state import copyStateValues

In [6]:
from pyfme.environment.gravity import VerticalConstant

In [7]:
environment = Environment()

In [8]:
controls_seq = {
    'delta_elevator': Constant(0),
    'delta_aileron': Constant(0),
    'delta_rudder': Constant(0),
    'delta_throttle': Constant(0)
}

In [9]:
system = RigidBodyQuat(aircraft, environment)

In [10]:
state = RigidBodyQuatState(np.array([0,0,0,1.,0,0,0,5.,0,0,0,0,0]))

In [ ]:
%prun sol = system.integrate(5, state, controls_seq, dt_eval=0.0005)

# Vary elevator input

In [ ]:
data = []
i=0
for de in np.linspace(-1,1,10):
    for w in np.linspace(-1,1,10):
        # update control sequence
        controls_seq['delta_elevator'] = Step(t_init=1, T=3, A=de, offset=0)
        
        # integrate equations of motion
        sol = system.integrate(5, state, controls_seq)
        d = sol.to_pandas()
        
        # compute controls
        for k in controls_seq:
            d[k] = controls_seq[k](d.time)
        
        # compute forces and moments
        controls = system.aircraft.get_controls(sol.time, controls_seq)
        conditions = system.environment.calculate_aero_conditions(sol)
        L, D, Y, l, m, n = system.aircraft._calculate_aero_forces_moments(conditions, sol, controls)
        d['L'] = L
        d['D'] = D
        d['Y'] = Y 
        d['l'] = l 
        d['m'] = m 
        d['n'] = n 
        
        # save
        d.to_csv('trajectory'+str(i)+'.csv', index=False)
        data.append(d)
        i+=1

# Vary initial condition

In [ ]:
##### ## /!\ Everything is cast to integer

In [12]:
data = []
i=0
for i in range(100):
    # update initial state value
    state.u = np.random.uniform(4,7)
    state.w = np.random.uniform(-.5,2)
    state.q = np.random.uniform(-.05,.05)

    # integrate equations of motion
    sol = system.integrate(5, state, controls_seq)
    d = sol.to_pandas()

    # compute controls
    for k in controls_seq:
        d[k] = controls_seq[k](d.time)

    # compute forces and moments
    controls = system.aircraft.get_controls(sol.time, controls_seq)
    conditions = system.environment.calculate_aero_conditions(sol)
    L, D, Y, l, m, n = system.aircraft._calculate_aero_forces_moments(conditions, sol, controls)
    d['L'] = L
    d['D'] = D
    d['Y'] = Y 
    d['l'] = l 
    d['m'] = m 
    d['n'] = n 

    # save
    d.to_csv('trajectory'+str(i)+'.csv', index=False)
    data.append(d)